# **Control SDP using the Tango devices**

Controlling Science Data Processor (SDP) using Tango devices allows to directly interact with the devices such as querying,
changing attributes and executing commands directly to Tango.

The Tango devices in the SDP are:
   - The SDP controller Tango device is designed to provide the overall control of the SDP. The commands it receives cause
     the other SDP services to be stopped or started, and its attributes report on the overall state of the system.
   - The SDP subarray Tango device is the principal means by which processing is initiated in the SDP.

If you get stuck you can look at the official documentation pages:
 - [SDP Integration](https://developer.skao.int/projects/ska-sdp-integration/en/latest/index.html)
 - [SDP Local Monitoring and Control](https://developer.skao.int/projects/ska-sdp-lmc/en/latest/index.html)
 - [SDP Processing Scripts](https://developer.skao.int/projects/ska-sdp-script/en/latest/)
 - [SDP on the Developer Portal](https://developer.skao.int/en/latest/projects/area/sdp.html)

If you still don't find the answer to your questions, contact us on Slack: #......


## 1. Import packages and set environments

Import all the required packages and define namespace and databaseds service.

In [ ]:
import os
from tango import DeviceProxy, EventType

# specify here the namespace to connect in this cluster
KUBE_NAMESPACE = "<update-with-ns!!!>"

# set the name of the databaseds service
DATABASEDS_NAME = "databaseds-tango-base"

# finally set the TANGO_HOST
os.environ["TANGO_HOST"] = f"{DATABASEDS_NAME}.{KUBE_NAMESPACE}.svc.cluster.local:10000"

## 2. Accessing the Tango Interface

Let's start by obtaining a handle of the subarray tango device:

In [ ]:
d = DeviceProxy('test-sdp/subarray/02')

Let's check the state of the device

In [ ]:
d.state()

The device should be in `OFF` state. This means the device is in inactive state

Let's check subarray observing state

In [ ]:
d.obsState

The above output would be `EMPTY`. This means no receive and real-time processing resources are assigned to the subarray

Going to set the device into its operational state

In [ ]:
d.On()
d.state()

The state should now be in `ON`. This means it has now been transitioned to operational state.

Now, we need to start the execution block using the `AssignResources` command. It takes an arguments which
contains configuration data in JSON format. The data are described by a schema which is versioned to support
evolution of the interfaces. The schema is specified in the argument with the interface keyword. The configuration
string to the command which defines scan types and processing blocks (real-time and batch).

Here is the configuration string for the execution block.

In [ ]:
import json
import random
from datetime import date

generator = "notebook"
today = date.today().strftime("%Y%m%d")
number = random.randint(0, 99999)

EXECUTION_BLOCK_ID = f"eb-{generator}-{today}-{number:05d}"
PROCESSING_BLOCK_ID = f"pb-{generator}-{today}-{number:05d}"

config = \
{
   "interface":"https://schema.skao.int/ska-sdp-assignres/0.3",
   "eb_id":f"{EXECUTION_BLOCK_ID}",
   "max_length":21600.0,
   "scan_types":[
      {
         "scan_type_id":"science",
         "reference_frame":"ICRS",
         "ra":"02:42:40.771",
         "dec":"-00:00:47.84",
         "channels":[
            {
               "count":13824,
               "count":5,
               "start":0,
               "stride":2,
               "freq_min":0.35e9,
               "freq_max":0.368e9,
               "link_map":[
                  [
                     0,
                     0
                  ],
                  [
                     200,
                     1
                  ],
                  [
                     744,
                     2
                  ],
                  [
                     944,
                     3
                  ]
               ]
            }
         ]
      },
      {
         "scan_type_id":"calibration",
         "reference_frame":"ICRS",
         "ra":"12:29:06.699",
         "dec":"02:03:08.598",
         "channels":[
            {
               "count":13824,
               "count":5,
               "start":0,
               "stride":2,
               "freq_min":0.35e9,
               "freq_max":0.368e9,
               "link_map":[
                  [
                     0,
                     0
                  ],
                  [
                     200,
                     1
                  ],
                  [
                     744,
                     2
                  ],
                  [
                     944,
                     3
                  ]
               ]
            }
         ]
      }
   ],
   "mccs": {
      "station_ids": [0, 1, 2, 3, 4, 5]
   },
   "processing_blocks":[
      {
         "pb_id": f"{PROCESSING_BLOCK_ID}",
         "workflow":{
            "kind":"realtime",
            "name":"vis-receive",
            "version":"0.5.1"
         },
         "parameters": {
            "plasmaEnabled": True,
            "transmission": {
               "channels_per_stream": 6912,
               "rate": "10416667"
            },
            "reception": {
               "layout": "http://127.0.0.1:80/model/default/ska1_low/layout",
               "schedblock": "/mnt/data/sb-test.json",
               "continuous_mode": True
            },
            "pvc": {
               "name": "receive-data"
            },
            "plasma_parameters": {
               "initContainers":[
                  {
                     "name":"existing-output-remover",
                     "image":"artefact.skao.int/ska-sdp-realtime-receive-modules:2.1.0",
                     "command":["rm", "-rf", "/mnt/data/output.ms"],
                     "volumeMounts":[
                        {
                           "mountPath":"/mnt/data",
                           "name":"receive-data"
                        }
                     ]
                  }
               ],
               "extraContainers":[
                  {
                     "name":"plasma-processor",
                     "image":"artefact.skao.int/ska-sdp-realtime-receive-modules:2.1.0",
                     "command":[
                        "plasma-mswriter",
                        "-s",
                        "/plasma/socket",
                        "--max_payloads",
                        "12",
                        "--use_plasma_ms",
                        "False",
                        "/mnt/data/output.ms"
                     ],
                     "volumeMounts":[
                        {
                           "name":"plasma-storage-volume",
                           "mountPath":"/plasma"
                        },
                        {
                           "mountPath":"/mnt/data",
                           "name":"receive-data"
                        }
                     ]
                  },
                  {
                     "name": "tmlite-server",
                     "image": "artefact.skao.int/ska-sdp-tmlite-server:0.3.0"
                  }
               ]
            }
         }
      }
   ]
}

config_eb = json.dumps(config)

The `AssignResources` command processes resources to the subarray and sets receive addresses.
The `obsState` will be in `RESOURCING` when the resources are being assigned and then will be set to `IDLE`
once the receive and real-time processing resources are assigned to the subarray as specified in the execution block.

In [ ]:
d.AssignResources(config_eb)

Once the command above is executed, need to wait until the `obsState` is set to `IDLE`.

In [ ]:
d.obsState

Let's check if the `receiveAddresses` attribute is set with the host addresses and ports for receiving visibilities.

In [ ]:
d.receiveAddresses

The `Configure` command configures scan type for the following scans. The argument it takes specifies the scan type.
It can declare new scan types to add the ones already defined for the execution block.

In [ ]:
d.Configure('{"interface": "https://schema.skao.int/ska-sdp-configure/0.3", "scan_type": "science"}')
d.obsState

The `obsState` will be transitioning from `CONFIGURING` when the scan type is being configured and will
be set to `READY` when the scan type is configured and the subarray is ready to scan.

The `Scan` command begins a scan of the configured type. The argument it takes specifies the scan ID.

In [ ]:
d.Scan('{"interface": "https://schema.skao.int/ska-sdp-scan/0.3", "scan_id": 1}')
d.obsState

The `obsState` will be set to `SCANNING` and it begins scanning.

To end the scan, run the following command.

In [ ]:
d.EndScan()
d.obsState

The `obsState` will be set to `READY`.

To clear the scan type, run the `End` command.

In [ ]:
d.End()
d.obsState

The `obsState` should be set to `IDLE`.

To end the execution block and to release all the real-time processing in the subarray run the following command.
This ends the real-time processing blocks and batch processing starts once when the resources are available.

In [ ]:
d.ReleaseResources()

In [ ]:
d.obsState

In this case the `obsState` will be transitioning from `RESOURCING` to `EMPTY` which means no real-time processing
resources are assigned to the subarray.

To set the device to inactive active, run the following command

In [ ]:
d.Off()
d.state()

In [ ]:
The state should be set to `OFF`.

More details about each of the SDP Subarray commands can be found here [here](https://developer.skao.int/projects/ska-sdp-lmc/en/latest/sdp_subarray.html)

More details about each of the SDP Subarray commands can be found here [here](https://developer.skao.int/projects/ska-sdp-lmc/en/latest/sdp_subarray.html)